In [6]:
%load_ext autotime
import pyspark as spark

import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.91 ms (started: 2021-10-11 20:06:09 +07:00)


In [16]:
sc = spark.SparkContext.getOrCreate()

sqlContext = spark.sql.SQLContext(sc)

bo_df = sqlContext.read.json('Data/Bombing_Operations.json.gz')
bo_df.printSchema()
bo_df.show()

root
 |-- AirCraft: string (nullable = true)
 |-- ContryFlyingMission: string (nullable = true)
 |-- MissionDate: string (nullable = true)
 |-- OperationSupported: string (nullable = true)
 |-- PeriodOfDay: string (nullable = true)
 |-- TakeoffLocation: string (nullable = true)
 |-- TargetCountry: string (nullable = true)
 |-- TimeOnTarget: double (nullable = true)
 |-- WeaponType: string (nullable = true)
 |-- WeaponsLoadedWeight: long (nullable = true)

+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|AirCraft| ContryFlyingMission|MissionDate|OperationSupported|PeriodOfDay|TakeoffLocation|TargetCountry|TimeOnTarget|          WeaponType|WeaponsLoadedWeight|
+--------+--------------------+-----------+------------------+-----------+---------------+-------------+------------+--------------------+-------------------+
|   EC-47|UNITED STATES OF ...| 1971-06-05|              null| 

In [15]:
ag_df = sqlContext.read.json('Data/Aircraft_Glossary.json.gz')
ag_df.printSchema()
ag_df.show()

root
 |-- AirCraft: string (nullable = true)
 |-- AirCraftName: string (nullable = true)
 |-- AirCraftType: string (nullable = true)

+--------+--------------------+--------------------+
|AirCraft|        AirCraftName|        AirCraftType|
+--------+--------------------+--------------------+
|     A-1|Douglas A-1 Skyra...|         Fighter Jet|
|    A-26|Douglas A-26 Invader|        Light Bomber|
|    A-37|Cessna A-37 Drago...|Light ground-atta...|
|     A-4|McDonnell Douglas...|         Fighter Jet|
|     A-5|North American A-...|          Bomber Jet|
|     A-6|Grumman A-6 Intruder|     Attack Aircraft|
|     A-7|  LTV A-7 Corsair II|     Attack Aircraft|
|  AC-119|Fairchild AC-119 ...|Military Transpor...|
|  AC-123|Fairchild C-123 P...|Military Transpor...|
|  AC-130|Lockheed AC-130 S...|Fixed wing ground...|
|   AC-47|Douglas AC-47 Spooky|Ground attack air...|
|    AH-1| Bell AH-1 HueyCobra|          Helicopter|
|     B-1| Rockwell B-1 Lancer|Heavy strategic b...|
|    B-52| B-52 St

In [21]:
from pyspark.sql.functions import desc
bo_df.groupBy('ContryFlyingMission').count().sort(desc('count')).collect()

[Row(ContryFlyingMission='UNITED STATES OF AMERICA', count=3708997),
 Row(ContryFlyingMission='VIETNAM (SOUTH)', count=622013),
 Row(ContryFlyingMission='LAOS', count=32777),
 Row(ContryFlyingMission='KOREA (SOUTH)', count=24469),
 Row(ContryFlyingMission='AUSTRALIA', count=12519)]

time: 8.54 s (started: 2021-10-11 20:52:40 +07:00)


In [23]:
bo_df.groupBy('ContryFlyingMission','MissionDate').count().sort(desc('count')).take(10)

[Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-06-07', count=3832),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-09-16', count=3755),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-09-18', count=3752),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-09-17', count=3706),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-05-11', count=3692),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-09-14', count=3646),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-06-16', count=3639),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-06-23', count=3630),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-06-11', count=3628),
 Row(ContryFlyingMission='UNITED STATES OF AMERICA', MissionDate='1968-09-19', count=3614)]

time: 10.4 s (started: 2021-10-11 20:54:00 +07:00)


In [46]:
bombardier = ag_df.select('AirCraft').filter(ag_df.AirCraftType.contains('omber')).collect()
bombardier_arr = []
for i in bombardier:
    bombardier_arr.append(i['AirCraft'])
bombardier_arr

['A-26',
 'A-5',
 'B-1',
 'B-52',
 'B-57',
 'B-66',
 'EB-66',
 'EKA-3',
 'F-100',
 'F-104',
 'F-105',
 'F-111',
 'F-4',
 'KA3',
 'RA-3',
 'RB-66',
 'RF-4',
 'A-3']

time: 143 ms (started: 2021-10-11 21:58:02 +07:00)


In [49]:
bo_df\
    .filter(bo_df.MissionDate =='1966-06-29')\
    .filter(bo_df.TargetCountry == 'NORTH VIETNAM')\
    .filter(bo_df.OperationSupported == 'ROLLING THUN')\
    .filter(bo_df.AirCraft.isin(bombardier_arr))\
    .groupBy('ContryFlyingMission')\
    .count()\
    .collect()

[Row(ContryFlyingMission='UNITED STATES OF AMERICA', count=159)]

time: 6.87 s (started: 2021-10-11 21:59:25 +07:00)


In [52]:
bo_df.filter(bo_df.TargetCountry.contains('VIETNAM'))\
    .groupby('AirCraft').count()\
    .sort(desc('count'))\
    .take(1)

[Row(AirCraft='F-4', count=500103)]

time: 11.5 s (started: 2021-10-11 22:02:22 +07:00)
